# ATMOSPHERE_1976

## Overview
The `ATMOSPHERE_1976` function provides the properties of the US Standard Atmosphere 1976 as a function of altitude above sea level. It returns key atmospheric properties such as temperature, pressure, density, viscosity, thermal conductivity, gravity, and speed of sound for a given elevation. This model is widely used in engineering and scientific applications for its empirical accuracy up to 86,000 meters and is based on extensive research by NASA and other agencies.

The model is valid from -610 meters (below sea level) up to 86,000 meters. It is a revision of the US 1962 atmosphere and is identical to the International Standard Atmosphere (ISA) and World Meteorological Organization (WMO) standard up to 32 km.

## Usage
To use this function in Excel:
```excel
=ATMOSPHERE_1976(Z, [dT])
```
- `Z` is required; `dT` is optional.

## Arguments
| Argument | Type  | Required | Description                                                        | Example |
|:-------- |:----- |:-------- |:-------------------------------------------------------------------|:------- |
| Z        | float | Required | Elevation above sea level [m]                                      | 5000    |
| dT       | float | Optional | Temperature difference from standard conditions [K]                | 0.0     |

## Returns
| Returns   | Type   | Description                                              | Example                |
|:----------|:------ |:--------------------------------------------------------|:-----------------------|
| T         | float  | Temperature of atmosphere at specified conditions [K]    | 255.675                |
| P         | float  | Pressure of atmosphere at specified conditions [Pa]      | 54048.29               |
| rho       | float  | Mass density at specified conditions [kg/m^3]            | 0.7364                 |
| H         | float  | Geopotential height [m]                                  | 4999.0                 |
| g         | float  | Acceleration due to gravity [m/s^2]                      | 9.7912                 |
| mu        | float  | Viscosity at specified conditions [Pa*s]                 | 1.628e-5               |
| k         | float  | Thermal conductivity [W/m/K]                             | 0.02273                |
| v_sonic   | float  | Speed of sound [m/s]                                     | 320.55                 |
| Error     | string | Error message if calculation fails                       | "Error: Invalid input" |

## Examples
### Calculating Atmospheric Properties at 5,000 m
| Argument | Value |
|:-------- |:----- |
| Z        | 5000  |
| dT       | 0.0   |

```excel
=ATMOSPHERE_1976(5000)
```
**Expected Output:**
| Returns | Value      |
|:------- |:---------- |
| T       | 255.675    |
| P       | 54048.29   |
| rho     | 0.7364     |
| H       | 4999.0     |
| g       | 9.7912     |
| mu      | 1.628e-5   |
| k       | 0.02273    |
| v_sonic | 320.55     |
| Error   | ""         |

### Calculating Properties at Sea Level with Temperature Offset
| Argument | Value |
|:-------- |:----- |
| Z        | 0     |
| dT       | 5.0   |

```excel
=ATMOSPHERE_1976(0, 5.0)
```
**Expected Output:**
| Returns | Value      |
|:------- |:---------- |
| T       | 293.15     |
| P       | 101325.0   |
| rho     | 1.225      |
| H       | 0.0        |
| g       | 9.80665    |
| mu      | 1.789e-5   |
| k       | 0.024      |
| v_sonic | 347.2      |
| Error   | ""         |

## Reference
See the `fluids` package [GitHub repository](https://github.com/CalebBell/fluids) and [documentation](https://fluids.readthedocs.io/index.html) for details.  We are grateful to the `fluids` package authors for their work.

In [ ]:
import micropip
await micropip.install('fluids')
from fluids.atmosphere import ATMOSPHERE_1976

def atmosphere_1976(Z, dT=0.0):
    """
    Calculate US Standard Atmosphere 1976 properties for a given altitude.

    Args:
        Z (float): Elevation above sea level [m].
        dT (float, optional): Temperature difference from standard conditions [K].

    Returns:
        list: [T, P, rho, H, g, mu, k, v_sonic, Error]
            T (float): Temperature [K]
            P (float): Pressure [Pa]
            rho (float): Density [kg/m^3]
            H (float): Geopotential height [m]
            g (float): Gravity [m/s^2]
            mu (float): Viscosity [Pa*s]
            k (float): Thermal conductivity [W/m/K]
            v_sonic (float): Speed of sound [m/s]
            Error (str): Error message if calculation fails
    """
    try:
        atm = ATMOSPHERE_1976(Z, dT)
        return [
            getattr(atm, "T", None),
            getattr(atm, "P", None),
            getattr(atm, "rho", None),
            getattr(atm, "H", None),
            getattr(atm, "g", None),
            getattr(atm, "mu", None),
            getattr(atm, "k", None),
            getattr(atm, "v_sonic", None),
            ""
        ]
    except Exception as e:
        return [None, None, None, None, None, None, None, None, f"Error: {str(e)}"]

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_atmosphere_1976_5000():
    result = atmosphere_1976(5000)
    assert isinstance(result, list)
    assert len(result) == 9
    assert result[0] is not None and result[1] is not None
    assert result[-1] == ""

def test_atmosphere_1976_sea_level_offset():
    result = atmosphere_1976(0, 5.0)
    assert isinstance(result, list)
    assert len(result) == 9
    assert result[0] is not None and result[1] is not None
    assert result[-1] == ""

def test_atmosphere_1976_invalid():
    result = atmosphere_1976("bad_input")
    assert isinstance(result, list)
    assert len(result) == 9
    assert result[-1].startswith("Error:")

ipytest.run()

In [ ]:
import gradio as gr

def gradio_input_converter(Z, dT):
    # Convert blank/None to None, else cast to float
    Z_val = None if Z is None or Z == "" else float(Z)
    dT_val = None if dT is None or dT == "" else float(dT)
    if dT_val is None:
        return atmosphere_1976(Z_val)
    return atmosphere_1976(Z_val, dT_val)

examples = [[5000, "0.0"], [0, "5.0"]]

demo = gr.Interface(
    fn=gradio_input_converter,
    inputs=[
        gr.Number(label="Z (Elevation above sea level) [m]", value=5000),
        gr.Textbox(label="dT (Temperature offset) [K]", placeholder="Optional", value="0.0"),
    ],
    outputs=[
        gr.Number(label="T [K]"),
        gr.Number(label="P [Pa]"),
        gr.Number(label="rho [kg/m^3]"),
        gr.Number(label="H [m]"),
        gr.Number(label="g [m/s^2]"),
        gr.Number(label="mu [Pa*s]"),
        gr.Number(label="k [W/m/K]"),
        gr.Number(label="v_sonic [m/s]"),
        gr.Textbox(label="Error"),
    ],
    examples=examples,
    description="Calculate US Standard Atmosphere 1976 properties for a given altitude. Returns temperature, pressure, density, geopotential height, gravity, viscosity, thermal conductivity, speed of sound, and error message if any.",
    flagging_mode='never',
)
demo.launch()